Ran on March 17th 5:00

In [1]:
import pandas as pd
import os
import cv2
import numpy as np
import json

In [2]:
def Detect_Office(Json,Office):

    NewList=Json['fields']
    Dict=list()
    for d in NewList:
        try:
            newDict={}
            newDict['inferText']=d['inferText']
            newDict['boundingPoly']=d['boundingPoly']
            Dict.append(newDict)
        except KeyError:
            continue

    res = [d
       for d in Dict 
       if (Office[0] == d['inferText']) or (Office == d['inferText']) or (Office[-2:][0] == d['inferText'])]

    if len(res)!=0:
        res = res[0]['boundingPoly']['vertices']
        Edge=max(int(d['x']) for d in res)
        return(Edge)
    else:
        return(None)

class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NpEncoder, self).default(obj)

### CLOVA FUNCTION ###
import requests
import uuid
import time
import json
import cv2
import base64

api_url = 'https://deelieyxuc.apigw.ntruss.com/custom/v1/1972/ebd01bcbf693d069817622e9839e20914143c7d0d8953eddee40e8b0af96c95b/general'
secret_key = 'S1NmVXpYZlJ0cGJ0ZEFRZXVlbkRkaHFReE9FcHNTQ0U='

def Clova(Year,Page):
    path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
    with open(path+"Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg",'rb') as f:
         file_data = f.read()

    request_json = {
            'images': [
                {
                    'format': 'jpg',
                    'name': 'demo',
                    'data':base64.b64encode(file_data).decode()}],
            'requestId': str(uuid.uuid4()),
            'version': 'V2',
            'timestamp': int(round(time.time() * 1000)),
            'lang':'ja'
            }
    payload = json.dumps(request_json).encode("UTF-8")
    headers = {'X-OCR-SECRET': secret_key,
              'Content-Type': 'application/json'}
    response = requests.request("POST", api_url, headers=headers, data = payload)
    Json=json.loads(response.text)['images'][0]
    
    return Json

In [3]:
Year='1935'
Showa='10'
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
os.chdir(path)
df = pd.read_csv(r'C:/Users/Keitaro Ninomiya/Box/Research Notes (keitaro2@illinois.edu)/Tokyo_Jobs/Processed_Data/Index/S'+Showa+'.csv')
df=df.drop(df.columns[0], axis=1)

file_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'+Year+'\\DataFrame.json'
with open(file_path, encoding="utf-8") as f:
    dta = json.loads(f.read())

In [4]:
n=0
Row  = df.iloc[n]
Page=int(Row["Page"])
Office=Row["Office"]
Dept=Row['Dept']
print(Office,Dept)
dta[Year][Dept][Office]={}
dta[Year][Dept][Office]["Starting_Page"]=Page

img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
#Convert to json via CLOVA
Json=Clova(Year,Page)

#Find X coordinate of 'Office'.
XCoord_Unit=Detect_Office(Json,Office)
dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
HH=img.shape[:2][0]
print(Office+ 'success')
cv2.line(img, (XCoord_Unit,0), (XCoord_Unit,HH), (255,0,0), 2)
cv2.imshow('pic',img)
cv2.waitKey(0)

秘書課 Admin
秘書課success


-1

In [5]:
#Test code| Version 2#
#Show Working office#
FailedList=[]
for n in range(1,len(df)):
    #Extract key info of office
    Row  = df.iloc[n]
    ExRow= df.iloc[n-1]

    Page=int(Row["Page"])
    Office=Row["Office"]
    Dept=Row['Dept']

    ExPage=int(ExRow["Page"])
    ExOffice=ExRow["Office"]
    ExDept=ExRow['Dept']

    ###Insert Starting page information to motherframe###
    dta[Year][Dept][Office]={}
    dta[Year][Dept][Office]["Starting_Page"]=Page
    print(dta[Year][Dept][Office])

    ###Collect Location information###
    ##Read image for first page##
    img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
    #Convert to json via CLOVA
    Json=Clova(Year,Page)

    #Find X coordinate of 'Office'.
    XCoord_Unit=Detect_Office(Json,Office)
    if XCoord_Unit==None:
        #Add to motherframe
        dta[str(Year)][Dept][Office]["Office_X1"]='NA'
        dta[str(Year)][ExDept][ExOffice]["Ending_Page"]='NA'
        dta[str(Year)][ExDept][ExOffice]["Office_X2"]='NA'
        dta[str(Year)][ExDept][ExOffice]["Page_Range"]='NA'
        print(Office+ 'failed')
        FailedList.append(list((Dept,Office)))
        continue
    else:
        dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
        dta[str(Year)][ExDept][ExOffice]["Ending_Page"]=Page
        dta[str(Year)][ExDept][ExOffice]["Office_X2"]=XCoord_Unit+10
        dta[str(Year)][ExDept][ExOffice]["Page_Range"]=list(range(ExPage,Page+1))       
        HH=img.shape[:2][0]
        print(Office+ 'success')

{'Starting_Page': 3}
職員課success
{'Starting_Page': 4}
文書課success
{'Starting_Page': 5}
庶務課success
{'Starting_Page': 6}
監察課success
{'Starting_Page': 7}
区政課success
{'Starting_Page': 8}
統計課success
{'Starting_Page': 10}
都市計画課failed
{'Starting_Page': 12}
主計課success
{'Starting_Page': 13}
公債課success
{'Starting_Page': 14}
収納課success
{'Starting_Page': 20}
経理課success
{'Starting_Page': 21}
地理課success
{'Starting_Page': 24}
会計課success
{'Starting_Page': 25}
庶務課success
{'Starting_Page': 25}
商工課success
{'Starting_Page': 27}
農魚課success
{'Starting_Page': 29}
権度課success
{'Starting_Page': 30}
庶務課success
{'Starting_Page': 31}
学務課failed
{'Starting_Page': 31}
社会教育課failed
{'Starting_Page': 34}
体育課success
{'Starting_Page': 38}
視学課success
{'Starting_Page': 42}
庶務課success
{'Starting_Page': 43}
保護課success
{'Starting_Page': 52}
福利課success
{'Starting_Page': 55}
職業課failed
{'Starting_Page': 59}
庶務課success
{'Starting_Page': 60}
衛生課success
{'Starting_Page': 63}
清掃課success
{'Starting_Page': 73}
公園課failed
{'Starting_Page':

In [6]:
FailRate=len(FailedList)/len(df)
if len(FailedList)/len(df)<0.2:
    print('Fantastic!! Success Rate is '+str(1-(len(FailedList)/len(df))))
else:
    print('残念...Success Rate is '+str(1-(len(FailedList)/len(df))))
DF=pd.read_csv('C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\Records.csv')
DF.loc[int(Year)-1934, 'Office'] = 1-FailRate

DF.to_csv('C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\Records.csv')

Fantastic!! Success Rate is 0.859375


In [7]:
DF

,Unnamed: 0,Year,Office,Unit,Position,Name
0,0,1934,NaN,NaN,NaN,Nino
1,1,1935,0.859375,NaN,NaN,Nino
2,2,1936,NaN,NaN,NaN,Yuya
3,3,1937,NaN,NaN,NaN,Yuya
4,4,1938,NaN,0.464286,NaN,Nino
5,5,1939,NaN,NaN,NaN,Yuya
6,6,1940,NaN,NaN,NaN,Nino
7,7,1941,NaN,NaN,NaN,Yuya
8,8,1942,NaN,NaN,NaN,NaN
9,9,1943,NaN,NaN,NaN,NaN


In [11]:
#Fixing Failed Offices
#Step1: Check for simple page assignment error
path="C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Raw_Data\\Splited\\"+Year+"\\"
for n in range(2,len(FailedList)):
    Dept=FailedList[n][0]
    Office=FailedList[n][1]
    print(Dept,Office)
    Page=df['Page'][(df['Office']==Office) & (df['Dept']==Dept)].tolist()[0]
    image=cv2.imread(path+"Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
    cv2.imshow('Image',image)
    cv2.waitKey(0)

教育局 社会教育課
社会局 職業課
保健局 公園課
土木局 庶務課
電気局 人事掛
電気局 庶務課
養育院 井之頭学校


In [12]:
json_object = json.dumps(dta, indent=4,
                        cls=NpEncoder)
save_path='C:\\Users\\Keitaro Ninomiya\\Box\\Research Notes (keitaro2@illinois.edu)\\Tokyo_Jobs\\Processed_Data\\'+str(Year)+'\\'
with open(save_path+"DataFrame.json", "w") as outfile:
    outfile.write(json_object)

In [32]:
df

,Office,Dept,Year,Page
0,秘書課,Admin,10,2
1,職員課,Admin,10,3
2,文書課,Admin,10,4
3,庶務課,監査局,10,5
4,監察課,監査局,10,6
...,...,...,...,...
59,医務課,養育院,10,148
60,会計課,養育院,10,149
61,井之頭学校,養育院,10,150
62,巣鴨分院,養育院,10,150


In [46]:
Fail = FailedList[1]
Dept=Fail[0]
Office=Fail[1]
print(Dept,Office)
df_Dept=df[df['Dept']==Dept]
df_Office=df_Dept[df_Dept['Office']==Office]
Page=int(df_Office['Page'])

Json=Clova(Year,Page)
[d['inferText'] for d in Json['fields']]

教育局 社会教育課


['性',
 '真',
 '同',
 '人工',
 '六上',
 '年一七〇〇',
 '九七',
 '五',
 '月七五',
 '事務員',
 '主事',
 '丹五五',
 '月六五',
 '拜人有',
 'を',
 '技手牽',
 '凡七五',
 '四八五',
 '社會教育課',
 '乗手',
 '/',
 '務',
 '告被助机',
 '市公隆福',
 '木長加佐山菊石鹽',
 '割',
 '下坂藤藤 岸 池 村 澤',
 '伸',
 '登',
 '太',
 '好 蛭 肚',
 '米',
 '眞 英',
 '郎一言介祐藏助治',
 '青年教育掛長費',
 '課長老五、',
 '月六五',
 '七下',
 '月七五',
 'に',
 '円 五 五',
 '六下',
 '學事長長中',
 '管理掛長大入八',
 '課長爵五、動六尾',
 '売',
 '教育品',
 '!',
 '土',
 '、七五',
 '大森、厚ケ谷、三〇五',
 '&',
 '田谷、一九二',
 '湖、"QQ→一七七',
 '(世田谷三七六3',
 '青田谷、健康、',
 '谷',
 '佐川',
 '木尾',
 '米',
 '厚自揚步驟 1將芋片白芝麻湯',
 '野',
 '矢',
 '池',
 '宮龍',
 '石澤手方',
 '宣',
 '田 勇次郎',
 '治',
 '唯',
 '七〇',
 '秀',
 '源一',
 '廣士',
 '智慧企業者 免費税金',
 '金光浩',
 '布',
 '眞',
 '雄',
 '彥',
 '雄',
 '-31-']

In [42]:
int(Page)

150

In [11]:
#Test code| Version 2#
#Decompose info into elements
Page=int(Row["Page"])
Office=Row["Office"]
print(Office)
ExPage=int(ExRow["Page"])
ExOffice=ExRow["Office"]


###Insert Starting page information to motherframe###
dta[Year][Dept][Office]={}
dta[Year][Dept][Office]["Starting_Page"]=Page
print(dta)

###Collect Location information###
##Read image for first page##
img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
#Convert to json via CLOVA
Json=Clova(Year,Page)

#Find X coordinate of 'Office'.
XCoord_Unit=Detect_Office(Json,Office)
#Add to motherframe
dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
dta[str(Year)][ExDept][ExOffice]["Ending_Page"]=Page
dta[str(Year)][ExDept][ExOffice]["Office_X2"]=XCoord_Unit+10
dta[str(Year)][ExDept][ExOffice]["Page_Range"]=list(range(ExPage,Page+1))
dta[str(Year)][Dept]

文書課
{'1934': {'Admin': {'文書課': {'Starting_Page': 3}, '職員課': {'Starting_Page': 2}, '庶務課': {'Ending_Page': 5, 'Office_X2': 160, 'Page_Range': [5]}, '監察課': {'Starting_Page': 5, 'Office_X1': 150}}, '中央卸売市場': {}, '保健局': {}, '土木局': {}, '教育局': {}, '水道局': {}, '港湾部（S8.11.1）': {}, '産業局': {}, '監査局': {'庶務課': {'Starting_Page': 5, 'Office_X1': 429}, '監察課': {'Starting_Page': 5, 'Office_X1': 150}}, '社会局': {}, '財務局': {}, '電気局(S8.11.6)': {}, '電気研究所（S8.11.18）': {}}}


{'文書課': {'Starting_Page': 3, 'Office_X1': 292},
 '職員課': {'Starting_Page': 2,
  'Ending_Page': 3,
  'Office_X2': 302,
  'Page_Range': [2, 3]},
 '庶務課': {'Ending_Page': 5, 'Office_X2': 160, 'Page_Range': [5]},
 '監察課': {'Starting_Page': 5, 'Office_X1': 150}}

In [5]:
#Test code| Version 2#
#Show Working office#
n=3

#Extract key info of office
Row  = df.iloc[n]
ExRow= df.iloc[n-1]
Row

Office    庶務課
Dept      監査局
Year        9
Page        5
Name: 3, dtype: object

In [6]:
Page=int(Row["Page"])
Dept=Row["Dept"]
Office=Row["Office"]
print(Office)
ExPage=int(ExRow["Page"])
ExDept=ExRow["Dept"]
ExOffice=ExRow["Office"]


###Insert Starting page information to motherframe###
dta[Year][Dept][Office]={}
dta[Year][Dept][Office]["Starting_Page"]=Page
print(dta)

###Collect Location information###
##Read image for first page##
img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
#Convert to json via CLOVA
Json=Clova(Year,Page)

#Find X coordinate of 'Office'.
XCoord_Unit=Detect_Office(Json,Office)
#Add to motherframe
dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
dta[str(Year)][ExDept][ExOffice]["Ending_Page"]=Page
dta[str(Year)][ExDept][ExOffice]["Office_X2"]=XCoord_Unit+10
dta[str(Year)][ExDept][ExOffice]["Page_Range"]=list(range(ExPage,Page+1))
dta

庶務課
{'1934': {'Admin': {'文書課': {'Starting_Page': 3, 'Office_X1': 292, 'Ending_Page': 5, 'Office_X2': 439, 'Page_Range': [3, 4, 5], 'Position': {'Manager': {'XLocation': 261, 'Page': 3, 'EndLocation': 230.0, 'EndPage': 3.0}, 'Leader': {'XLocation': 230, 'Page': 3, 'EndLocation': 469.0, 'EndPage': 4.0}, 'Admin': {'XLocation': 469, 'Page': 4, 'EndLocation': 302.0, 'EndPage': 4.0, 'Data': [{'Name': '淺野元三郎', 'Wage': '幸'}, {'Name': '五人店TE', 'Wage': '月七'}, {'Name': '小川有道', 'Wage': '𠮷'}, {'Name': '志賀漆', 'Wage': 'お'}, {'Name': '京山重長', 'Wage': '1'}, {'Name': '天主任人江博房', 'Wage': '万九和'}, {'Name': '各个長下條装', 'Wage': 'カト'}, {'Name': '和久田賞', 'Wage': '上'}, {'Name': '木村五', 'Wage': 'あや'}, {'Name': '長坂番', 'Wage': '幸'}, {'Name': '宮尾D20', 'Wage': '吉'}, {'Name': '正版 松留', 'Wage': '七七'}, {'Name': '鍜田國信', 'Wage': '七上'}, {'Name': '部見安二郎', 'Wage': 'ス'}, {'Name': '高畑稔', 'Wage': '奉'}, {'Name': '長友英太郎', 'Wage': '円1'}, {'Name': '荒井千', 'Wage': '六上'}, {'Name': 'M成七七郎', 'Wage': '大'}]}, 'Outsource': {'XLocation': 302, 'Pa

{'1934': {'Admin': {'文書課': {'Starting_Page': 3,
    'Office_X1': 292,
    'Ending_Page': 5,
    'Office_X2': 439,
    'Page_Range': [3, 4, 5],
    'Position': {'Manager': {'XLocation': 261,
      'Page': 3,
      'EndLocation': 230.0,
      'EndPage': 3.0},
     'Leader': {'XLocation': 230,
      'Page': 3,
      'EndLocation': 469.0,
      'EndPage': 4.0},
     'Admin': {'XLocation': 469,
      'Page': 4,
      'EndLocation': 302.0,
      'EndPage': 4.0,
      'Data': [{'Name': '淺野元三郎', 'Wage': '幸'},
       {'Name': '五人店TE', 'Wage': '月七'},
       {'Name': '小川有道', 'Wage': '𠮷'},
       {'Name': '志賀漆', 'Wage': 'お'},
       {'Name': '京山重長', 'Wage': '1'},
       {'Name': '天主任人江博房', 'Wage': '万九和'},
       {'Name': '各个長下條装', 'Wage': 'カト'},
       {'Name': '和久田賞', 'Wage': '上'},
       {'Name': '木村五', 'Wage': 'あや'},
       {'Name': '長坂番', 'Wage': '幸'},
       {'Name': '宮尾D20', 'Wage': '吉'},
       {'Name': '正版 松留', 'Wage': '七七'},
       {'Name': '鍜田國信', 'Wage': '七上'},
       {'Name': '部見安二郎', 

In [7]:
#Test code| Version 2#
#Show Working office#
n=4

#Extract key info of office
Row  = df.iloc[n]
ExRow= df.iloc[n-1]
Row

Office    監察課
Dept      監査局
Year        9
Page        5
Name: 4, dtype: object

In [20]:
Page=int(Row["Page"])
Office=Row["Office"]
print(Office)
ExPage=int(ExRow["Page"])
ExOffice=ExRow["Office"]


###Insert Starting page information to motherframe###
dta[Year][Dept][Office]={}
dta[Year][Dept][Office]["Starting_Page"]=Page
print(dta)

###Collect Location information###
##Read image for first page##
img=cv2.imread("Page"+"{:03d}".format(Page)+"\\"+"Page"+"{:03d}".format(Page)+".jpg")
#Convert to json via CLOVA
Json=Clova(Year,Page)

#Find X coordinate of 'Office'.
XCoord_Unit=Detect_Office(Json,Office)
#Add to motherframe
dta[str(Year)][Dept][Office]["Office_X1"]=XCoord_Unit
dta[str(Year)][ExDept][ExOffice]["Ending_Page"]=Page
dta[str(Year)][ExDept][ExOffice]["Office_X2"]=XCoord_Unit+10
dta[str(Year)][ExDept][ExOffice]["Page_Range"]=list(range(ExPage,Page+1))
dta

監察課
{'1934': {'Admin': {'文書課': {'Starting_Page': 3, 'Office_X1': 292, 'Ending_Page': 5, 'Office_X2': 439, 'Page_Range': [3, 4, 5]}, '職員課': {'Starting_Page': 2, 'Ending_Page': 3, 'Office_X2': 302, 'Page_Range': [2, 3]}, '庶務課': {'Ending_Page': 5, 'Office_X2': 160, 'Page_Range': [5]}, '監察課': {'Starting_Page': 5, 'Office_X1': 150}}, '中央卸売市場': {}, '保健局': {}, '土木局': {}, '教育局': {}, '水道局': {}, '港湾部（S8.11.1）': {}, '産業局': {}, '監査局': {'庶務課': {'Starting_Page': 5, 'Office_X1': 429}, '監察課': {'Starting_Page': 5}}, '社会局': {}, '財務局': {}, '電気局(S8.11.6)': {}, '電気研究所（S8.11.18）': {}}}


{'1934': {'Admin': {'文書課': {'Starting_Page': 3,
    'Office_X1': 292,
    'Ending_Page': 5,
    'Office_X2': 439,
    'Page_Range': [3, 4, 5]},
   '職員課': {'Starting_Page': 2,
    'Ending_Page': 3,
    'Office_X2': 302,
    'Page_Range': [2, 3]},
   '庶務課': {'Ending_Page': 5, 'Office_X2': 160, 'Page_Range': [5]},
   '監察課': {'Starting_Page': 5, 'Office_X1': 150}},
  '中央卸売市場': {},
  '保健局': {},
  '土木局': {},
  '教育局': {},
  '水道局': {},
  '港湾部（S8.11.1）': {},
  '産業局': {},
  '監査局': {'庶務課': {'Starting_Page': 5, 'Office_X1': 429},
   '監察課': {'Starting_Page': 5, 'Office_X1': 150}},
  '社会局': {},
  '財務局': {},
  '電気局(S8.11.6)': {},
  '電気研究所（S8.11.18）': {}}}

In [121]:
n=10
Dept=FailedList[n][0]
Office=FailedList[n][1]
print(Dept,Office)
Page=df['Page'][(df['Office']==Office) & (df['Dept']==Dept)].tolist()[0]
Json=Clova(Year,Page)


水道局 会計課
